# Lower bound inspection

Search for differences

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import yaml
import subprocess

In [ ]:
def get_d(instance):
    p = subprocess.run(f"../cmake-build-release/lower_bound_inspection \
                       --input {instance}".split(), capture_output=True)
    d = yaml.safe_load("\n".join(p.stdout.decode("utf-8").split("\n")[1:]))
    
    d["adjacency"] = np.array(d["adjacency"]["values"])
    d["costs"] = np.array(d["costs"]["values"])
    d["packing_cover"] = np.array(d["packing_cover"]["values"])
    d["relaxation"] = np.array(d["relaxation"]["values"])

    d["packing"] = list(map(tuple, d["packing"]))
    d["packing_min_cost_vertex_pairs"] = list(map(tuple, d["packing_min_cost_vertex_pairs"]))
    
    return d
    

In [ ]:
instances = ["bio-nr-1854-size-14.graph",
 "bio-nr-1480-size-15.graph",
 "bio-nr-979-size-79.graph",
 "bio-nr-931-size-52.graph",
 "bio-nr-269-size-86.graph",
 "bio-nr-875-size-45.graph"]

In [ ]:
instance = instances[3]
d = get_d(f"../data/bio/{instance}")

In [ ]:
def plot_instance(d, instance):
    G = nx.Graph(d["adjacency"])
    pos = nx.nx_pydot.graphviz_layout(G)

    def subgraph_to_vertex_pairs(vertices):
        return [(u, v) for u in vertices for v in vertices if u < v]

    def plot_graph(vertex_pairs=[], subgraphs=[], title=None, path=None):
        fig, ax = plt.subplots(figsize=(4, 4))
        ax.set_aspect("equal")
        ax.set_title(title)
        
        #ax.set_xticks([])
        #ax.set_yticks([])
        ax.axis('off')

        vertex_pairs = list(tuple(e) for e in vertex_pairs)

        nx.draw_networkx_nodes(G, pos, with_labels=False, node_size=10)
        nx.draw_networkx_edges(G, pos, edgelist=set(G.edges()) - set(vertex_pairs))
        nx.draw_networkx_edges(G, pos, edgelist=vertex_pairs, edge_color="C0", width=5, alpha=0.5)

        for S, color in zip(subgraphs, (f"C{i}" for i in range(len(subgraphs)))):
            nx.draw_networkx_edges(G, pos, edgelist=subgraph_to_vertex_pairs(S), edge_color=color, width=2)
        
        
        fig.tight_layout()
        
        if path is not None:
            plt.savefig(path)
        
        plt.show()
    
    
    vertex_pairs_used_by_packing = d["packing_min_cost_vertex_pairs"]
    vertex_pairs_used_by_relaxation = list((u, v) for (u, v) in zip(*np.where(d["relaxation"] > 0)) if u < v)
    
    print(f"Instance: {instance}")
    print(f"Relaxation: {d['relaxation_lb']} {vertex_pairs_used_by_relaxation}")    
    print(f"Values taken: {np.unique(d['relaxation'])}")
    print(f"Packing: {d['packing_lb']} {vertex_pairs_used_by_packing}")
    print(f"{d['packing']}")

    plot_graph(title="Original graph",
               path=f"{instance}.original.pdf")
    plot_graph(vertex_pairs_used_by_packing,
               title="Packing vertex pairs",
               path=f"{instance}.packing-vertex-pairs.pdf")
    plot_graph(vertex_pairs_used_by_relaxation,
               title="Relaxation vertex pairs ($x_{uv} > 0$)",
               path=f"{instance}.relaxation-vertex-pairs.pdf")
    plot_graph(subgraphs=d["packing"],
               title="Subgraphs in the packing",
               path=f"{instance}.packing-subgraphs.pdf")

for instance in instances[3:4]:
    d = get_d(f"../data/bio/{instance}")
    plot_instance(d, instance)

In [ ]:


print("Instance")
fig, axes = plt.subplots(ncols=2, figsize=(9, 4))

axes[0].set_title("Adjacency")
axes[0].matshow(d["adjacency"])

axes[1].set_title("Editing costs")
m = axes[1].matshow(d["costs"])
fig.colorbar(m)

for ax in axes:
    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.tight_layout()
plt.show()


print("Relaxation")
print(f"Values taken: {np.unique(d['relaxation'])}")

fig, ax = plt.subplots(figsize=(5, 4))

ax.set_title("Relaxation values")
m = ax.matshow(d["relaxation"])
fig.colorbar(m)

ax.set_xticklabels([])
ax.set_yticklabels([])

fig.tight_layout()
plt.show()



fig, axes = plt.subplots(ncols=2, figsize=(8, 4))

axes[0].set_title("Packing cover")
axes[0].matshow(d["packing_cover"])

packing_min_cost_vertex_pairs = np.zeros_like(d["adjacency"])
for (u, v) in d["packing_min_cost_vertex_pairs"]:
    packing_min_cost_vertex_pairs[(u, v)] = 1
    packing_min_cost_vertex_pairs[(v, u)] = 1
axes[1].set_title("...")
axes[1].matshow(packing_min_cost_vertex_pairs)

for ax in axes:
    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.tight_layout()
plt.show()

In [ ]:
ax.set_xlabel()

In [ ]:
def plot_activation_hist(x, costs):
    y = (x[x > 0] * costs[x > 0]).flatten()
    plt.hist(y, bins=20)
    plt.show()

plot_activation_hist(d["relaxation"], d["costs"])
plot_activation_hist(packing_min_cost_vertex_pairs, d["costs"])